1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

--2022-11-11 10:39:38--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2022-11-11 10:39:39--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc777c3e8b4c61f7613e432612b4.dl.dropboxusercontent.com/cd/0/inline/Bwj3IqOD_YF6dfmdwYHc46hBc-7vkKEfXhSkFg9p37c17mQopdqDrP_AkoVsImstM4SV5GZg_Wtn3qFiRPfXluYrKrcKCFYxR7eLHJnpq4lq5oZY4B0hOTPVnKy4mzup_HTRmkVcqlLaRwP7R4l_Q_o9emFP7bONjyOxGMnu0hCfdQ/file# [following]
--2022-11-11 10:39:40--  https://uc777c3e8b4c61f7613e432612b4.dl.dropboxusercontent.com/cd/0/inline/Bwj3IqOD_YF6dfmdwYHc46hBc-7vkKEfXhSkFg9p37c17mQopdqDrP_AkoVsImstM4SV5GZg_Wtn3qFiRPfXluYrKrcKCFYxR

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [14]:
import pandas as pd # standard naming convention
import numpy as np
file_name = "./data/data_000637.txt"
data = pd.read_csv(file_name)
data
print(data.shape)
N=np.random.randint(10000,data.shape[0])
df=pd.DataFrame(data[:N])
df

(1310720, 6)


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
1084044,1,0,61,3869209608,1789,21
1084045,1,1,139,3869209608,1809,0
1084046,1,1,5,3869209608,1803,22
1084047,1,0,45,3869209608,1806,10


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [57]:

x=df.min()
print(x)
#print(np.where(df["BX_COUNTER"] == x['BX_COUNTER']))
print('The number of BX in a ORBIT is: ',x['BX_COUNTER'])
#print(df.loc[4073])


HEAD                      1
FPGA                      0
TDC_CHANNEL               1
ORBIT_CNT        3869200167
BX_COUNTER                0
TDC_MEAS                  0
Absolute Time            25
dtype: int64
The number of BX in a ORBIT is:  0


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

In [43]:
df['Absolute Time']=pd.Series(25*(df.index+1) , index=df.index)

df
#[df.loc[i,'Absolute Time'] = 25*df.index[i+1] for i in range(df.shape[0])]

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,Absolute Time
0,1,0,123,3869200167,2374,26,25
1,1,0,124,3869200167,2374,27,50
2,1,0,63,3869200167,2553,28,75
3,1,0,64,3869200167,2558,19,100
4,1,0,64,3869200167,2760,25,125
...,...,...,...,...,...,...,...
1084044,1,0,61,3869209608,1789,21,27101125
1084045,1,1,139,3869209608,1809,0,27101150
1084046,1,1,5,3869209608,1803,22,27101175
1084047,1,0,45,3869209608,1806,10,27101200


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [53]:
print(df.groupby('TDC_CHANNEL')['ORBIT_CNT'].max()[:3])

TDC_CHANNEL
1    3869209608
2    3869209608
3    3869209607
Name: ORBIT_CNT, dtype: int64


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.